<a href="https://colab.research.google.com/github/badrinarayanan02/LLM/blob/main/2348507_LLMLab8(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Finetuning BERT LLM for Sentiment Analysis


Going to Finetune DistillBERT llm model using Transformers and Tensforflow



DistillBERT - It is a smaller version of BERT. It tries to mimic the pretrained model BERT. It has less computations.

Loading the Dataset

In [51]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import DistilBertTokenizerFast
import tensorflow as tf
import numpy as np
from transformers import TFDistilBertForSequenceClassification, Trainer, TFTrainingArguments, DistilBertTokenizerFast, TFDistilBertForSequenceClassification

In [4]:
df=messages = pd.read_csv('/content/SMSSpamCollection..txt', sep='\t',
                           names=["label", "message"])
df.head()


,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [5]:
df.shape

(5572, 2)

Converting into Independent and Dependent Features

In [6]:
X = list(df['message'])

In [7]:
y = list(df['label'])

In [8]:
X[:5]

['Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...',
 'Ok lar... Joking wif u oni...',
 "Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's",
 'U dun say so early hor... U c already then say...',
 "Nah I don't think he goes to usf, he lives around here though"]

In [9]:
y[:5]

['ham', 'ham', 'spam', 'ham', 'ham']

In [10]:
y_series = pd.Series(y)

Converting the dependent feature to a numerical value (boolean)

In [11]:
dummies = pd.get_dummies(y_series,drop_first=True)

In [12]:
dummies

,spam
0,False
1,False
2,True
3,False
4,False
...,...
5567,True
5568,False
5569,False
5570,False


Converting into a list

In [13]:
y = list(pd.get_dummies(dummies,drop_first=True)['spam'])

In [14]:
y[:5]

[False, False, True, False, False]

Splitting the Data

In [15]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [16]:
X_train[:10]

['Reply to win £100 weekly! Where will the 2006 FIFA World Cup be held? Send STOP to 87239 to end service',
 'Hello. Sort of out in town already. That . So dont rush home, I am eating nachos. Will let you know eta.',
 'How come guoyang go n tell her? Then u told her?',
 'Hey sathya till now we dint meet not even a single time then how can i saw the situation sathya.',
 'Orange brings you ringtones from all time Chart Heroes, with a free hit each week! Go to Ringtones & Pics on wap. To stop receiving these tips reply STOP.',
 'Sitting in mu waiting for everyone to get out of my suite so I can take a shower',
 'Finish liao... U?',
 'URGENT! Your Mobile No 07808726822 was awarded a £2,000 Bonus Caller Prize on 02/09/03! This is our 2nd attempt to contact YOU! Call 0871-872-9758 BOX95QU',
 'Probably not, still going over some stuff here',
 'Wah lucky man... Then can save money... Hee...']

Installing Transformers

In [17]:
pip install transformers

After Installing Transformers

1) Call the pretrained model

2) Call the tokenizer (For specific model specific tokenizer is there)

3) Convert the encodings to a data object

Loading the pretrained model

In [18]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Performing Encodings

In [19]:
train_encodings = tokenizer(X_train, truncation = True, padding = True)
test_encodings = tokenizer(X_test, truncation = True, padding=True)

In [20]:
train_encodings[:10]

[Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=238, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing]),
 Encoding(num_tokens=238

Converting these encodings into a dataset objects using tensorflow

In [21]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    y_train
)).batch(16)

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    y_test
)).batch(16)

In [22]:
train_dataset

<_BatchDataset element_spec=({'input_ids': TensorSpec(shape=(None, 238), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(None, 238), dtype=tf.int32, name=None)}, TensorSpec(shape=(None,), dtype=tf.bool, name=None))>

This is the format that is required for sequence classifications.

Initializing the Pretrained Model

In [44]:
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [24]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics = ['accuracy']
)

Used keras because of TFTrainer deprecation in Tensorflow.

In [25]:
history = model.fit(train_dataset, validation_data = test_dataset, epochs = 2)

Epoch 1/2
279/279 [==============================] - 138s 386ms/step - loss: 0.0697 - accuracy: 0.9796 - val_loss: 0.0242 - val_accuracy: 0.9946
Epoch 2/2
279/279 [==============================] - 111s 397ms/step - loss: 0.0209 - accuracy: 0.9957 - val_loss: 0.0388 - val_accuracy: 0.9919


In [26]:
loss, accuracy = model.evaluate(test_dataset)
print(f"Test Accuracy: {accuracy:.4f}")

70/70 [==============================] - 6s 84ms/step - loss: 0.0388 - accuracy: 0.9919
Test Accuracy: 0.9919


In [27]:
predictions = model.predict(test_dataset)

70/70 [==============================] - 7s 81ms/step


In [28]:
predictions

TFSequenceClassifierOutput(loss=None, logits=array([[ 3.77279  , -4.2008176],
       [ 3.450275 , -3.8596172],
       [ 3.5966132, -4.050575 ],
       ...,
       [ 3.7607718, -4.203236 ],
       [ 3.6102257, -4.024377 ],
       [ 3.6451924, -4.0971446]], dtype=float32), hidden_states=None, attentions=None)

These are logits score. Logits - Raw scores output by the model.

Conversion of Logits to Probabilities

In [29]:
logits = predictions.logits
probabilities = tf.nn.softmax(logits,axis=1)
predicted_labels = tf.argmax(probabilities, axis=-1).numpy()

In [30]:
y_test_np = np.array(y_test)
print('Predicted Labels:',predicted_labels[:20])
print("True Labels:", y_test_np[:20])

Predicted Labels: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]
True Labels: [False False False False False False False False False False False False
 False False  True False False False False False]


In [49]:
model.save_pretrained("sentiment_analysis_model")
tokenizer.save_pretrained("sentiment_analysis_model")

('sentiment_analysis_model/tokenizer_config.json',
 'sentiment_analysis_model/special_tokens_map.json',
 'sentiment_analysis_model/vocab.txt',
 'sentiment_analysis_model/added_tokens.json',
 'sentiment_analysis_model/tokenizer.json')

# Developing a Python script to load the fine-tuned model and tokenizer

In [52]:
model_path = "/content/sentiment_analysis_model"

tokenizer = DistilBertTokenizerFast.from_pretrained(model_path)

model = TFDistilBertForSequenceClassification.from_pretrained(model_path)

Some layers from the model checkpoint at /content/sentiment_analysis_model were not used when initializing TFDistilBertForSequenceClassification: ['dropout_39']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /content/sentiment_analysis_model and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
def predict_sentiment(messages):

    encodings = tokenizer(messages, truncation=True, padding=True, return_tensors="tf")

    logits = model(encodings).logits

    probabilities = tf.nn.softmax(logits, axis=1)

    predicted_labels = tf.argmax(probabilities, axis=1).numpy()

    labels = ["ham", "spam"]
    predicted_sentiments = [labels[label] for label in predicted_labels]

    return predicted_sentiments

new_messages = [
    "Free entry in 2 a weekly competition to win FA Cup final tkts 21st May 2005.",
    "I'm going to be late for the meeting today.",
    "Congratulations! You have won a $1000 Walmart gift card. Click here to claim."
]

predicted_sentiments = predict_sentiment(new_messages)
for message, sentiment in zip(new_messages, predicted_sentiments):
    print(f"Message: '{message}' - Sentiment: {sentiment}")

Message: 'Free entry in 2 a weekly competition to win FA Cup final tkts 21st May 2005.' - Sentiment: ham
Message: 'I'm going to be late for the meeting today.' - Sentiment: ham
Message: 'Congratulations! You have won a $1000 Walmart gift card. Click here to claim.' - Sentiment: spam


# Conclusion

Finetuned the DistilBert LLM model. Gave a custom dataset that includes spam or ham. The model will analyse the sentiment of the message whether it is ham or spam. Used transformers to do all the required operations. Tensorflow is used for the conversion of encodings to dataset object. Used keras because I faced an issue with TFTrainer in tensorflow, since it is deprecated. At last loaded the fine tuned model and and it gave a good response.